In [112]:
#@formatter:off
import re
import numpy as np
import pandas
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

pandas.options.display.max_rows = 100
%matplotlib inline
sns.set_theme(style='whitegrid')
#@formatter:on

In [129]:
def which_contains(self, regexp):
    if not isinstance(regexp, str):
        regexp = iterable_to_regexp(regexp)
    return self.loc[self.str.contains(regexp)]


def iterable_to_regexp(entries):
    return '|'.join([re.escape(entry) for entry in entries])


pd.Series.which_contains = which_contains

In [2]:
train_df = pd.read_csv('data/titanic_train.csv')
test_df = pd.read_csv('data/titanic_test.csv')
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [168]:
def get_tags(column, tags):
    output = pd.Series(index=column.index, dtype=object)
    for tag in tags:
        output[column.str.contains(tag)] = tag
    return output

list_of_list_of_names = train_df.Name.str.split().values
words = [word for lst in list_of_list_of_names for word in lst]
titles = pd.Series(words).which_contains('\\.').str.replace('\\.', '').unique()
print(titles)


['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'L' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']


In [169]:
# for title in titles:
    # print(train_df.loc[train_df['Name'].str.contains(title), 'Survived'])
# output = pd.Series(index=train_df.index, dtype=object)
# print(train_df.query('Survived == 1').Ticket)
# output.loc[train_df.query('Survived == 1').index] = train_df.query('Survived == 1').Ticket
# output.loc[train_df.query('Survived == 0').index] = train_df.query('Survived == 0').Ticket
# print(output)
# train_df.Name.which_contains(titles)
train_df['Title'] = get_tags(train_df.Name, titles)

[int, object]